In [1]:
#import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from numba import njit
from tqdm import tqdm
tqdm.pandas()
import concurrent.futures

In [2]:

MAX_NB_FRAMES = 900
DATA_FOLDER = "/mnt/e/sign-lang-data-2"

#print(all_col)


In [3]:
#this notebook is for the purpose of testing a data input pipeline

In [4]:
def load_data(filename):
    #filename = filename.numpy().decode("utf-8")

    max_nb_frames = MAX_NB_FRAMES
    import numpy as np
    import pandas as pd
    from pathlib import Path
    from tqdm import tqdm

    seq_label = pd.read_csv(os.path.join(DATA_FOLDER, "train", str(filename).replace("/", "_") + ".csv"), sep=";")
    seq_label = seq_label[["sequence_id", "phrase"]]

    parquet = pd.read_parquet(os.path.join(DATA_FOLDER, filename), engine='pyarrow')

    all_col = list(parquet.columns)

    parquet_group = parquet.groupby("sequence_id")

    x_face = [col for col in all_col if "x_face" in col]
    y_face = [col for col in all_col if "y_face" in col]
    z_face = [col for col in all_col if "z_face" in col]

    x_left_hand = [col for col in all_col if "x_left_hand" in col]
    y_left_hand = [col for col in all_col if "y_left_hand" in col]
    z_left_hand = [col for col in all_col if "z_left_hand" in col]

    x_right_hand = [col for col in all_col if "x_right_hand" in col]
    y_right_hand = [col for col in all_col if "y_right_hand" in col]
    z_right_hand = [col for col in all_col if "z_right_hand" in col]

    x_pose = [col for col in all_col if "x_pose" in col]
    y_pose = [col for col in all_col if "y_pose" in col]
    z_pose = [col for col in all_col if "z_pose" in col]

    stored_data = np.zeros((len(parquet_group),max_nb_frames, 543, 3))

    # crerate a tf tensor with shape (len(parquet_group), 1)


    for step, (name, sequence) in enumerate(parquet_group):
        for sub_step, (sub_name, timestep) in enumerate(sequence.groupby("frame")):
            stored_data[step, sub_step, :468, 0] = timestep[x_face].values
            stored_data[step, sub_step, :468, 1] = timestep[y_face].values
            stored_data[step, sub_step, :468, 2] = timestep[z_face].values

            stored_data[step, sub_step, 468:468 + 21, 0] = timestep[x_left_hand].values
            stored_data[step, sub_step, 468:468 + 21, 1] = timestep[y_left_hand].values
            stored_data[step, sub_step, 468:468 + 21, 2] = timestep[z_left_hand].values

            stored_data[step, sub_step, 468 + 21:468 + 21*2, 0] = timestep[x_right_hand].values
            stored_data[step, sub_step, 468 + 21:468 + 21*2, 1] = timestep[y_right_hand].values
            stored_data[step, sub_step, 468 + 21:468 + 21*2, 2] = timestep[z_right_hand].values

            stored_data[step, sub_step, 468 + 21*2:468 + 21*2 + 33, 0] = timestep[x_pose].values
            stored_data[step, sub_step, 468 + 21*2:468 + 21*2 + 33, 1] = timestep[y_pose].values
            stored_data[step, sub_step, 468 + 21*2:468 + 21*2 + 33, 2] = timestep[z_pose].values

    return stored_data, seq_label.values[:, 1]


@njit
def fill_data(stored_data, x_face, y_face, z_face, x_left_hand, y_left_hand, z_left_hand, x_right_hand, y_right_hand, z_right_hand, x_pose, y_pose, z_pose, sub_step):

    stored_data[sub_step, :468, 0] = x_face
    stored_data[sub_step, :468, 1] = y_face
    stored_data[sub_step, :468, 2] = z_face

    stored_data[sub_step, 468:468 + 21, 0] = x_left_hand
    stored_data[sub_step, 468:468 + 21, 1] = y_left_hand
    stored_data[sub_step, 468:468 + 21, 2] = z_left_hand

    stored_data[sub_step, 468 + 21:468 + 21*2, 0] = x_right_hand
    stored_data[sub_step, 468 + 21:468 + 21*2, 1] = y_right_hand
    stored_data[sub_step, 468 + 21:468 + 21*2, 2] = z_right_hand

    stored_data[sub_step, 468 + 21*2:468 + 21*2 + 33, 0] = x_pose
    stored_data[sub_step, 468 + 21*2:468 + 21*2 + 33, 1] = y_pose
    stored_data[sub_step, 468 + 21*2:468 + 21*2 + 33, 2] = z_pose

def create_npy(filename):
    #filename = filename.numpy().decode("utf-8")

    max_nb_frames = MAX_NB_FRAMES
    import numpy as np
    import pandas as pd
    from tqdm import tqdm
    import time
    from pathlib import Path

    start = time.time()
    parquet = pd.read_parquet(os.path.join(DATA_FOLDER, filename), engine='pyarrow')
    end = time.time()
    #print("time to read parquet file : ", end - start)

    all_col = list(parquet.columns)

    start = time.time()
    parquet_group = parquet.groupby("sequence_id")
    end = time.time()
    #print("time to group parquet file : ", end - start)

    x_face = [col for col in all_col if "x_face" in col]
    y_face = [col for col in all_col if "y_face" in col]
    z_face = [col for col in all_col if "z_face" in col]

    x_left_hand = [col for col in all_col if "x_left_hand" in col]
    y_left_hand = [col for col in all_col if "y_left_hand" in col]
    z_left_hand = [col for col in all_col if "z_left_hand" in col]

    x_right_hand = [col for col in all_col if "x_right_hand" in col]
    y_right_hand = [col for col in all_col if "y_right_hand" in col]
    z_right_hand = [col for col in all_col if "z_right_hand" in col]

    x_pose = [col for col in all_col if "x_pose" in col]
    y_pose = [col for col in all_col if "y_pose" in col]
    z_pose = [col for col in all_col if "z_pose" in col]

    #stored_data = np.zeros((len(parquet_group),max_nb_frames, 543, 3))

    # crerate a tf tensor with shape (len(parquet_group), 1)

    def process_group(name_sequence_pair):
        name, sequence = name_sequence_pair
        if os.path.exists(os.path.join(DATA_FOLDER, "npy_train_landmarks", str(filename).replace("/", "_").replace(".parquet", "") , str(name).replace("/", "_") + ".npy")):
            return None, None
        stored_data = np.zeros((max_nb_frames, 543, 3))
        max_frame = 0
        for sub_step, (sub_name, timestep) in enumerate(sequence.groupby("frame")):
            fill_data(
                stored_data,
                timestep[x_face].values, timestep[y_face].values, timestep[z_face].values,
                timestep[x_left_hand].values, timestep[y_left_hand].values, timestep[z_left_hand].values,
                timestep[x_right_hand].values, timestep[y_right_hand].values, timestep[z_right_hand].values,
                timestep[x_pose].values, timestep[y_pose].values, timestep[z_pose].values,
                sub_step
            )
            max_frame = sub_step
        Path(os.path.join(DATA_FOLDER, "npy_train_landmarks", str(filename).replace("/", "_").replace(".parquet", ""))).mkdir(exist_ok=True)

        np.save(
            os.path.join(DATA_FOLDER, "npy_train_landmarks", str(filename).replace("/", "_").replace(".parquet", ""),
                             str(name).replace("/", "_") + ".npy"), stored_data[:max_frame + 1, :, :]
                )

    with concurrent.futures.ThreadPoolExecutor(max_workers=16) as executor:
        list(executor.map(process_group, parquet_group))



In [5]:
train = pd.read_csv(os.path.join(DATA_FOLDER, "train.csv"))
train.columns

Index(['path', 'file_id', 'sequence_id', 'participant_id', 'phrase'], dtype='object')

In [6]:
train.iloc[0]["path"]

'train_landmarks/5414471.parquet'

In [7]:
#train["path"].drop_duplicates().progress_apply(create_npy)

In [8]:
#@njit()
#def optimaze_concatenate(arrays, axis=0):
#    return np.concatenate(arrays, axis=axis)


def load_npy(npy_path, phrase ):

    max_nb_frames = MAX_NB_FRAMES


    filename = npy_path.numpy().decode("utf-8")
    inputs = np.load(filename)

    if inputs.shape[0] > max_nb_frames:
        inputs = inputs[:max_nb_frames, :, :]
    else:
        inputs = np.concatenate([inputs, np.zeros((max_nb_frames - inputs.shape[0], 543, 3))], axis=0)


    return inputs, phrase.numpy().decode("utf-8")

In [25]:
import json
import tensorflow as tf
import keras_nlp
import keras

with open("monBytePairTokenizer.json", "r") as f:
    js_file = json.load(f)
# %%
vocab = js_file["model"]["vocab"]
merge = js_file["model"]["merges"]
# %%
tokenizer = keras_nlp.tokenizers.BytePairTokenizer(vocab, merge)

train["npy_path"] = train[["sequence_id", "path"]].apply(
    lambda x: os.path.join(DATA_FOLDER, "npy_train_landmarks", x["path"].replace(".parquet", "").replace("/", "_") ,str(x["sequence_id"]) + ".npy"),
    axis=1
)

train_dataset = tf.data.Dataset.from_tensor_slices((train["npy_path"].iloc[:50], train["phrase"].iloc[:50]))

# load only the npy
train_dataset = train_dataset.map(
    lambda npy_path, phrase: tf.py_function(func=load_npy, inp=[npy_path, phrase], Tout=(tf.float32, tf.string)),
    num_parallel_calls=tf.data.AUTOTUNE)


train_dataset = train_dataset.map(
    lambda inputs, label: (inputs, tokenizer.tokenize(label)),
    num_parallel_calls=tf.data.AUTOTUNE)

# add mask  of 4 to the output
def add_mask(inputs, label):
    label = label.numpy()
    shape = label.shape
    new_label = np.full((64), 4)
    new_label[:shape[1]] = label[0]
    return inputs, new_label

train_dataset = train_dataset.map(
    lambda inputs, label: tf.py_function(func=add_mask, inp=[inputs, label], Tout=(tf.float32, tf.int32)),
    num_parallel_calls=tf.data.AUTOTUNE)

# convert to one hot
train_dataset = train_dataset.map(
    lambda inputs, label: (inputs, tf.one_hot(label, depth=tokenizer.vocabulary_size())),
    num_parallel_calls=tf.data.AUTOTUNE)

#train_dataset = train_dataset.batch(1)
for step, (inputs, label) in enumerate(train_dataset):
    print(inputs.shape)
    print(label.shape)


2023-07-02 15:22:58.214859: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_12' with dtype resource
	 [[{{node Placeholder/_12}}]]
2023-07-02 15:22:58.215121: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype int64 and shape [8]
	 [[{{node Placeholder/_5}}]]


(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3)
(64, 30000)
(900, 543, 3

In [10]:
label = tf.constant([[1, 2, 3]])
label = tf.concat([label[0], tf.zeros((64 - label.shape[1],), dtype=tf.int32)], axis=0)

tf.one_hot(label, depth=tokenizer.vocabulary_size())

<tf.Tensor: shape=(64, 30000), dtype=float32, numpy=
array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [16]:
train["npy_path"]

0        /mnt/e/sign-lang-data-2/npy_train_landmarks/tr...
1        /mnt/e/sign-lang-data-2/npy_train_landmarks/tr...
2        /mnt/e/sign-lang-data-2/npy_train_landmarks/tr...
3        /mnt/e/sign-lang-data-2/npy_train_landmarks/tr...
4        /mnt/e/sign-lang-data-2/npy_train_landmarks/tr...
                               ...                        
67203    /mnt/e/sign-lang-data-2/npy_train_landmarks/tr...
67204    /mnt/e/sign-lang-data-2/npy_train_landmarks/tr...
67205    /mnt/e/sign-lang-data-2/npy_train_landmarks/tr...
67206    /mnt/e/sign-lang-data-2/npy_train_landmarks/tr...
67207    /mnt/e/sign-lang-data-2/npy_train_landmarks/tr...
Name: npy_path, Length: 67208, dtype: object

2023-07-02 10:29:15.870287: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_6' with dtype resource
	 [[{{node Placeholder/_6}}]]
2023-07-02 10:29:15.870665: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_9' with dtype string
	 [[{{node Placeholder/_9}}]]
2023-07-02 10:29:16.640051: W tensorflow/core/framework/op_kernel.cc:1818] INVALID_ARGUMENT: ValueError: Shape (63,) must have rank 2
Traceback (most recent call last):

  File "/home/dev/miniconda3/envs/sign-lang/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 265, in __call__
    return func(device, token, args)

  F

InvalidArgumentError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} ValueError: Shape (63,) must have rank 2
Traceback (most recent call last):

  File "/home/dev/miniconda3/envs/sign-lang/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 265, in __call__
    return func(device, token, args)

  File "/home/dev/miniconda3/envs/sign-lang/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 143, in __call__
    outputs = self._call(device, args)

  File "/home/dev/miniconda3/envs/sign-lang/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 150, in _call
    ret = self._func(*args)

  File "/home/dev/miniconda3/envs/sign-lang/lib/python3.10/site-packages/tensorflow/python/ops/script_ops.py", line 809, in wrapped_func
    out = func(*structured_inp)

  File "/home/dev/miniconda3/envs/sign-lang/lib/python3.10/site-packages/tensorflow/python/autograph/impl/api.py", l